# Importações


In [41]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from fuzzywuzzy import process
from enum import Enum


FERIADOS = pd.read_csv("../assets/feriados.csv")

### Types


In [42]:
# cSpell: disable
class BSColorsEnum(Enum):

    DANGER_COLOR = "#dc3545"

    WARNING_COLOR = "#ffc107"

    SUCCESS_COLOR = "#198754"

    GREY_500_COLOR = "#adb5bd"

    GREY_600_COLOR = "#6c757d"

    GREY_700_COLOR = "#495057"

    GREY_800_COLOR = "#343a40"

    GREY_900_COLOR = "#212529"

    PRIMARY_COLOR = "#0d6efd"

    SECONDARY_COLOR = "#6c757d"

    INFO_COLOR = "#0dcaf0"

    GRAY_COLOR = "#adb5bd"

    TEAL_COLOR = "#20c997"

    ORANGE_COLOR = "#fd7e14"

    INDIGO_COLOR = "#6610f2"

    PINK_COLOR = "#d63384"

    PURPLE_COLOR = "#6f42c1"

    GREY_400_COLOR = "#ced4da"

    SPACE_CADET_COLOR = "#282f44"

    BLUE_DELFT_COLOR = "#0d6efd"


class IndicatorType(Enum):
    PERFORMANCE = "performance"
    REPAIR = "reparo"
    EFFICIENCY = "eficiencia"


def get_color(value, max_value):
    """
    Retorna uma cor hexadecimal com base no valor fornecido e no valor máximo.

    Parâmetros:
    value (float): O valor para o qual a cor será calculada.
    max_value (float): O valor máximo possível.

    Retorna:
    str: Uma cor hexadecimal correspondente ao valor fornecido.
    """

    # Cria um mapa de cores que vai do vermelho ao verde
    cmap = plt.get_cmap("RdYlGn")

    # Normaliza o valor para um número entre 0 e 1
    normalized_value = float(value) / max_value

    # Obtém a cor correspondente do mapa de cores
    rgba_color = cmap(normalized_value)

    # Converte a cor RGBA para uma string de cor hexadecimal
    hex_color = (
        f"#{int(rgba_color[0]*255):02x}{int(rgba_color[1]*255):02x}{int(rgba_color[2]*255):02x}"
    )

    return hex_color

# Database


## Conexão com o banco de dados


In [43]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__database_totvsdb = getenv("PYMSSQL_DATABASE_TOTVSDB")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def get_connection_totvsdb(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database_totvsdb};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            conexao_totvsdb = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", pool_size=2)
            return conexao_totvsdb
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

## Leitura do banco de dados


In [44]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

    def get_totvsdb_data(self, query: str) -> pd.DataFrame:
        """
        Retrieves data from the TotvsDB database using the provided SQL query.

        Args:
            query (str): The SQL query to execute.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the retrieved data.

        Raises:
            Exception: If an error occurs while retrieving the data.

        """
        connection = None
        try:
            connection = self.get_connection_totvsdb()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_totvsdb_query(
        self, select: str, table: str, join: str = None, where: str = None, orderby: str = None
    ) -> str:
        """
        Creates a SQL query string for querying the TOTVS database.

        Args:
            select (str): The SELECT clause of the query.
            table (str): The table name to query.
            join (str, optional): The JOIN clause of the query. Defaults to None.
            where (str, optional): The WHERE clause of the query. Defaults to None.
            orderby (str, optional): The ORDER BY clause of the query. Defaults to None.

        Returns:
            str: The SQL query string.
        """
        query = f"SELECT {select} FROM {table}"

        if join:
            query += f" {join}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Query para o banco de dados


In [45]:
# // database/get_data.py
# cSpell: disable=invalid-name
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> tuple:
        """
        Realiza a leitura dos dados do banco de dados.
        Retorna na ordem: df_occ, df_info, df_cadastro
        """

        # Dia de hoje
        now = pd.to_datetime("today")

        # Encontrando primeiro dia do mês atual
        first_day = now.replace(day=1)

        # Mantendo apenas a data
        first_day = first_day.strftime("%Y-%m-%d")

        # Query para leitura dos dados de ocorrência
        query_ihm = self.db_read.create_automacao_query(
            table="maquina_ihm",
            where=f"data_registro >= '{first_day}'",
        )

        query_info = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha,"
            " (SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica,"
            " t1.status,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " t1.data_registro,"
            " t1.hora_registro"
            " FROM "
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro >= '{first_day}'"
            " ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
        )

        query_production = (
            "SELECT * "
            "FROM ( "
            "SELECT "
            "(SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica, "
            "(SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha, "
            "t1.maquina_id, "
            "t1.turno, "
            "t1.status, "
            "t1.contagem_total_ciclos as total_ciclos, "
            "t1.contagem_total_produzido as total_produzido, "
            "t1.data_registro, "
            "t1.hora_registro, "
            "ROW_NUMBER() OVER ( "
            "PARTITION BY t1.data_registro, t1.turno, t1.maquina_id "
            "ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
            ") AS rn "
            "FROM AUTOMACAO.dbo.maquina_info t1 "
            ") AS t "
            f" WHERE rn = 1 AND data_registro >= '{first_day}' AND hora_registro > '00:01'"
            " ORDER BY data_registro DESC, linha"
        )

        print("========== Baixando dados do DB ==========")

        # Leitura dos dados
        df_ihm = self.db_read.get_automacao_data(query_ihm)
        df_info = self.db_read.get_automacao_data(query_info)
        df_info_production = self.db_read.get_automacao_data(query_production)

        # Verificando se os dados foram lidos corretamente
        if df_ihm.empty or df_info.empty or df_info_production.empty:
            print("====== Erro na leitura dos dados ======")
            return None, None, None

        print("Ok...")

        return df_ihm, df_info, df_info_production


get_data = GetData()
df_ihm, df_info, df_info_production = get_data.get_data()

========== Baixando dados do DB ==========


Ok...


## Testes de saída do banco de dados


In [46]:
df_ihm.head(20)

,recno,linha,maquina_id,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro,hora_registro
0,15,4,TMF012,Parada Programada,,Parada Planejada,Sem Produção,,002357,2024-04-24,11:28:20.666666
1,16,4,TMF012,Parada Programada,,Parada Planejada,Backup,,002357,2024-04-24,14:47:36.466666
2,17,0,TMF004,Fluxo,Recheadora,Falta de Pasta,Batida de pasta interrompida pela qualidade,,002357,2024-04-26,09:11:56.196666
3,18,1,TMF003,Parada Programada,,Parada Planejada,Refeição,,002357,2024-04-26,11:29:24.796666


In [47]:
df_info.head(20)

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF004,9.0,1,true,VES,416.0,406.0,2024-04-26,16:22:56.626666
1,TMF014,8.0,1,false,VES,2510.0,0.0,2024-04-26,16:22:55.620000
2,TMF006,7.0,1,true,VES,452.0,450.0,2024-04-26,16:22:54.620000
3,TMF001,6.0,1,false,VES,0.0,0.0,2024-04-26,16:22:53.620000
4,TMF002,5.0,1,true,VES,262.0,168.0,2024-04-26,16:22:52.620000
5,TMF012,4.0,1,false,VES,0.0,0.0,2024-04-26,16:22:51.616666
6,TMF015,3.0,1,true,VES,522.0,516.0,2024-04-26,16:22:50.616666
7,TMF011,2.0,1,true,VES,270.0,210.0,2024-04-26,16:22:49.616666
8,TMF003,1.0,1,true,VES,476.0,468.0,2024-04-26,16:22:48.616666
9,TMF004,9.0,1,true,VES,378.0,368.0,2024-04-26,16:20:56.620000


In [48]:
df_info_production.head(20)

,fabrica,linha,maquina_id,turno,status,total_ciclos,total_produzido,data_registro,hora_registro,rn
0,1,1.0,TMF003,MAT,true,6912.0,6862.0,2024-04-26,15:58:48.550000,1
1,1,1.0,TMF003,NOT,true,8574.0,8554.0,2024-04-26,07:58:16.833333,1
2,1,1.0,TMF003,VES,true,476.0,468.0,2024-04-26,16:22:48.616666,1
3,1,2.0,TMF011,NOT,true,5512.0,5412.0,2024-04-26,07:58:17.833333,1
4,1,2.0,TMF011,VES,true,270.0,210.0,2024-04-26,16:22:49.616666,1
5,1,2.0,TMF011,MAT,true,6108.0,5990.0,2024-04-26,15:58:49.550000,1
6,1,3.0,TMF015,NOT,true,7946.0,7876.0,2024-04-26,07:58:18.836666,1
7,1,3.0,TMF015,VES,true,522.0,516.0,2024-04-26,16:22:50.616666,1
8,1,3.0,TMF015,MAT,true,9482.0,9356.0,2024-04-26,15:58:50.550000,1
9,1,4.0,TMF012,MAT,false,0.0,0.0,2024-04-26,15:58:51.553333,1


# Limpeza de dados e análise exploratória


## Análise de dados - Clean Data


In [65]:
# service/clean_data.py


# cSpell: disable=invalid-name
class CleanData:
    """
    Essa classe é responsável por limpar os dados lidos do banco de dados.
    """

    def __init__(self, df_ihm, df_info, df_info_production):
        self.df_ihm = df_ihm
        self.df_info = df_info
        self.df_info_production = df_info_production

    def __clean_basics(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados básicos de um DataFrame.
        """

        # Remove valores duplicados
        df = df.drop_duplicates()

        # Remove linhas com valores nulos
        df = df.dropna(subset=["maquina_id", "data_registro", "hora_registro"])

        # Converte a coluna 'hora_registro' para string e remove os milissegundos
        df["hora_registro"] = df["hora_registro"].astype(str).str.split(".").str[0]

        # Converte as colunas 'data_registro' e 'hora_registro' para datetime
        df["data_registro"] = pd.to_datetime(df["data_registro"])
        df["hora_registro"] = pd.to_datetime(df["hora_registro"], format="%H:%M:%S").dt.time

        # Substitui os valores NaN por 0 antes de converter para int
        df["linha"] = df["linha"].fillna(0).astype(int)

        # Coluna linha para int
        df["linha"] = df["linha"].astype(int)

        # Remover onde a 'linha' é 0
        df = df[df["linha"] != 0]

        return df

    def clean_data(self) -> tuple:
        """
        Limpa os dados lidos do banco de dados.
        Retorna na ordem: df_ihm, df_info, df_info_production
        """

        print("========== Limpando dados ==========")

        # Limpando os dados(removendo linhas duplicadas ou nulas em informações importantes)
        self.df_ihm = self.__clean_basics(self.df_ihm)
        self.df_info = self.__clean_basics(self.df_info)
        self.df_info_production = self.__clean_basics(self.df_info_production)

        print("Ok...")

        return self.df_ihm, self.df_info, self.df_info_production


clean_data = CleanData(df_ihm, df_info, df_info_production)
df_ihm_cleaned, df_info_cleaned, df_info_production_cleaned = clean_data.clean_data()

========== Limpando dados ==========
Ok...


In [66]:
df_ihm_cleaned.head(20)

,recno,linha,maquina_id,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro,hora_registro
0,15,4,TMF012,Parada Programada,,Parada Planejada,Sem Produção,,002357,2024-04-24,11:28:20
1,16,4,TMF012,Parada Programada,,Parada Planejada,Backup,,002357,2024-04-24,14:47:36
3,18,1,TMF003,Parada Programada,,Parada Planejada,Refeição,,002357,2024-04-26,11:29:24


In [68]:
df_info_cleaned.head(20)

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF004,9,1,true,VES,416.0,406.0,2024-04-26,16:22:56
1,TMF014,8,1,false,VES,2510.0,0.0,2024-04-26,16:22:55
2,TMF006,7,1,true,VES,452.0,450.0,2024-04-26,16:22:54
3,TMF001,6,1,false,VES,0.0,0.0,2024-04-26,16:22:53
4,TMF002,5,1,true,VES,262.0,168.0,2024-04-26,16:22:52
5,TMF012,4,1,false,VES,0.0,0.0,2024-04-26,16:22:51
6,TMF015,3,1,true,VES,522.0,516.0,2024-04-26,16:22:50
7,TMF011,2,1,true,VES,270.0,210.0,2024-04-26,16:22:49
8,TMF003,1,1,true,VES,476.0,468.0,2024-04-26,16:22:48
9,TMF004,9,1,true,VES,378.0,368.0,2024-04-26,16:20:56


In [67]:
df_info_production_cleaned.head(20)

,fabrica,linha,maquina_id,turno,status,total_ciclos,total_produzido,data_registro,hora_registro,rn
0,1,1,TMF003,MAT,true,6912.0,6862.0,2024-04-26,15:58:48,1
1,1,1,TMF003,NOT,true,8574.0,8554.0,2024-04-26,07:58:16,1
2,1,1,TMF003,VES,true,476.0,468.0,2024-04-26,16:22:48,1
3,1,2,TMF011,NOT,true,5512.0,5412.0,2024-04-26,07:58:17,1
4,1,2,TMF011,VES,true,270.0,210.0,2024-04-26,16:22:49,1
5,1,2,TMF011,MAT,true,6108.0,5990.0,2024-04-26,15:58:49,1
6,1,3,TMF015,NOT,true,7946.0,7876.0,2024-04-26,07:58:18,1
7,1,3,TMF015,VES,true,522.0,516.0,2024-04-26,16:22:50,1
8,1,3,TMF015,MAT,true,9482.0,9356.0,2024-04-26,15:58:50,1
9,1,4,TMF012,MAT,false,0.0,0.0,2024-04-26,15:58:51,1
